In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import multiprocessing
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk

In [2]:
df = pd.read_csv('data/fin-mk.txt', sep='\t')
df.head()

,finnish,macedonian
0,chaplinin poika,чарли чаплин во детето
1,"kuva hymyll, ja ehk kyyneleen kera.","слика со насмевка и можеби, солза."
2,hyvntekevisyyssairaala,добротворна болница
3,"nainen, jonka synti oli itiys.",жената чии грев беше мајчинството.
4,mies.,мажот.


In [3]:
X = df['finnish'].values
y = df['macedonian'].values

In [4]:
# fi_tok = [word_tokenize(sent) for sent in X]
df["finnish_token"] = df["finnish"].fillna("").map(nltk.word_tokenize)
df["macedonian_token"] = df["macedonian"].fillna("").map(nltk.word_tokenize)

In [5]:
fin_tok = df["finnish_token"].values
mk_tok = df["macedonian_token"].values

In [6]:
fi_clean = []
temp = []
for i in fin_tok:
    for j in i:
        if j[0] == '-' and len(j) > 1:
            print('da')
            temp.append(j[1:])
        elif j[-1] == '-' and len(j) > 1:
            print('ne')
            temp.append(j[:-1])
        else:
            temp.append(j)
    fi_clean.append(temp)
    temp = []

In [145]:
mk_clean = []
temp = []
for i in mk_tok:
    for j in i:
        if j[0] == '-' and len(j) > 1:
            temp.append(j[1:])
        elif j[-1] == '-' and len(j) > 1:
            temp.append(j[:-1])
        else:
            temp.append(j)
    mk_clean.append(temp)
    temp = []

In [7]:
model_mk = Word2Vec(mk_tok, size=300, window=5, min_count=1, iter=10, workers=multiprocessing.cpu_count())

In [8]:
model_fi = Word2Vec(fin_tok, size=300, window=5, min_count=1, iter=10, workers=multiprocessing.cpu_count())

In [9]:
words = list(model_fi.wv.vocab)
for word in words:
    if word == 'ihmiskudos':
        print(word)

ihmiskudos


In [10]:
model_mk['мажот']

/work/modules/Ubuntu/14.04/amd64/common/anaconda3/latest/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.36566305, -0.11955065, -0.61124164, -1.8410972 , -0.26028472,
       -0.44562432, -0.19170971,  0.87342924,  0.47943276,  0.12454691,
       -0.23919004,  0.21835719,  0.30332863, -0.29319543, -0.73331344,
        1.3667077 ,  0.6276063 , -0.87395364, -0.9301904 , -0.6717533 ,
       -0.38831457,  0.04456605, -0.08683205,  0.35810176,  0.7418917 ,
        0.36847687,  0.21121079,  0.394285  ,  0.9477648 , -1.0800874 ,
        0.00867191, -1.5411788 ,  0.28096277, -0.6304994 ,  0.04411799,
       -0.95954365, -0.17963345,  0.66231245,  0.25188512, -0.2029736 ,
        0.8979835 , -0.1751848 ,  0.22954947, -1.0121516 ,  0.8345671 ,
       -0.7511575 , -0.10996038,  0.61484367, -0.42362297, -1.0690339 ,
       -0.12424053,  0.4563241 ,  0.56175476,  0.45372677, -0.7052854 ,
       -0.14764154, -0.30449826, -0.4085369 , -0.1750655 , -0.73473305,
       -0.31553453,  0.66920114,  0.5406773 ,  0.50583893, -0.75967664,
        1.1977465 , -0.2211826 , -1.1034344 ,  0.33568642,  0.70

In [11]:
wv = model_mk.wv
wv.similar_by_word('понеделник')

[('вторник', 0.851332426071167),
 ('петок', 0.8361653089523315),
 ('четврток', 0.8212425708770752),
 ('10:00', 0.7581230401992798),
 ('среда', 0.7532303333282471),
 ('8:00', 0.7406070232391357),
 ('7:00', 0.7258443832397461),
 ('11:00', 0.71864914894104),
 ('сабота', 0.7167373895645142),
 ('9:00', 0.7105333805084229)]

In [12]:
model_mk.save('data/embeddings/mk_embeddings.bin')
model_fi.save('data/embeddings/fi_embeddings.bin')